Obtain data form Coinranking API   

In [2]:
import requests,json
import pandas as pd

coin_id = 1 # 1 is BTC, 2 is ETH, 4 is BCH
time_range = "1y"
price_currency = "EUR"
response = requests.get("https://api.coinranking.com/v1/public/coin/%s/history/%s?base=%s" % (coin_id, time_range, price_currency))
response

<Response [200]>

Dump response to JSON file

In [3]:
import os

os.makedirs('responses', exist_ok=True)
file = open('responses/coin%d-history%s-currency%s.json' % (coin_id, time_range, price_currency),'w+')
file.write(response.text)
file.close()


Load response in pandas data frame

In [4]:
response_json = json.loads(response.text)['data']['history']
data_frame = pd.DataFrame(response_json)
data_frame['price'] = pd.to_numeric(data_frame['price'])
data_frame['timestamp'] = pd.to_datetime(data_frame['timestamp'], unit='ms').dt.date
data_sequence = data_frame.groupby('timestamp').mean()['price']
data_sequence

timestamp
2018-09-02    6115.234563
2018-09-03    6237.371389
2018-09-04    6253.482447
2018-09-05    6324.652042
2018-09-06    6152.730976
                 ...     
2019-08-28    9059.787395
2019-08-29    8616.555789
2019-08-30    8667.161586
2019-08-31    8742.433461
2019-09-01    8831.975072
Name: price, Length: 365, dtype: float64

Normalize data

In [5]:
chunk_size = 30
chunks = []
for i in range(len(data_sequence) - chunk_size+1):
    chunks.append(data_sequence[i:i+chunk_size])

price_change_sequence = []
for chunk in chunks:
    # TODO: Rethink technique, add float casting
    price_change_sequence.append([(price / chunk[0]) - 1 for price in chunk])
len(price_change_sequence)

336

Split data into training and test sets

In [6]:
import numpy as np

price_matrix = np.array(price_change_sequence)
train_set_size = int(len(price_matrix) * 0.8)
train_set = price_matrix[:train_set_size, :]
np.random.shuffle(train_set)

x_train_set, y_train_set = train_set[:train_set_size, :-1], train_set[:train_set_size, -1]
x_test_set, y_test_set = price_matrix[train_set_size:, :-1], price_matrix[train_set_size:, -1]

x_train_set = np.reshape(x_train_set, (x_train_set.shape[0], x_train_set.shape[1], 1))
x_test_set = np.reshape(x_test_set, (x_test_set.shape[0], x_test_set.shape[1], 1))

len(x_train_set), len(x_test_set)

(268, 68)

Build LSTM network

In [7]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation


lstm = Sequential()
lstm.add(LSTM(units=chunk_size, return_sequences=True, input_shape=(None, 1)))
lstm.add(Dense(units=32,activation='linear'))
lstm.add(LSTM(units=chunk_size, return_sequences=False))
lstm.add(Dense(units=1,activation='linear'))
lstm.compile(optimizer='rmsprop',loss='mean_squared_error')

Using TensorFlow backend.


Train network

In [8]:
import time
start_time = time.time()
lstm.fit(x=x_train_set,
          y=y_train_set,
          batch_size=2,
          epochs=20,
          validation_split=0.05)
end_time = time.time()
processing_time = end_time - start_time
processing_time

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 254 samples, validate on 14 samples
Epoch 1/20
254/254 [==============================] - 8s 30ms/step - loss: 0.0101 - val_loss: 0.0016
Epoch 2/20
254/254 [==============================] - 6s 25ms/step - loss: 0.0049 - val_loss: 0.0026
Epoch 3/20
254/254 [==============================] - 7s 27ms/step - loss: 0.0038 - val_loss: 0.0011
Epoch 4/20
254/254 [==============================] - 7s 27ms/step - loss: 0.0034 - val_loss: 7.5493e-04
Epoch 5/20
254/254 [==============================] - 7s 27ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 6/20
254/254 [==============================] - 7s 29ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 7/20
254/254 [==============================] - 7s 27ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 8/20
254/254 [==============================] - 7s 26ms/step - loss: 0.0020 - val_loss: 7.8324e-04
Epoch 9/20
254/254 [=======================

139.58026313781738

Save network model

In [8]:
os.makedirs('model', exist_ok=True)
lstm.save('model/coin%d-history%s-currency%s.h5' % (coin_id, time_range, price_currency))

Load trained model from file

In [ ]:
lstm.load('model/coin%d-history%s-currency%s.h5' % (coin_id, time_range, price_currency))

Predict

In [9]:
predictions = lstm.predict(x_test_set, batch_size=2)

Denormalize predictions 

In [10]:
price_matrix = np.array(chunks)
denormalized_x_test = price_matrix[train_set_size:,:-1]
denormalized_y_test = price_matrix[train_set_size:,-1]
denormalized_predictions = []
predictions = np.reshape(predictions, (predictions.shape[0]))
for index in range(len(predictions)):
    pred = (predictions[index]+1)* denormalized_x_test[index][0]
    denormalized_predictions.append(pred)
denormalized_predictions = np.array(denormalized_predictions)

Visualize results

In [ ]:
import matplotlib.pyplot as plt

plot_data_frame = pd.DataFrame({'actual_values': denormalized_y_test, 'predictions'}) 
ax = plt.gca()

df.plot(kind='line',x='name',y='actual_values',ax=ax)
df.plot(kind='line',x='name',y='predictions', color='red', ax=ax)

plt.show()